#### Bibliotecas e Base de Dados

In [4]:
import pandas as pd
import os
import math
from statsmodels.tsa.seasonal import STL
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score

%matplotlib inline
import matplotlib.pyplot as plt
# Carregar o dataframe tratado
df = pd.read_csv("base_tratada_lingualab.csv")


In [5]:
df

,ID_Cliente,Motivo,Education,Cidade,Renda_anual,Aulas,NPS,Num_Aulas,Ano_de_nascimento,Banheiros_na_casa,Quartos_na_casa,motivo_na,renda_na,Quartos_Banheiros,faixa_idade
0,0,Viagem,Graduação,São Paulo,58138.000000,Ingles,4,19,1983,3,4,False,False,4Q_3B,1980s
1,1,Trabalho,Graduação,Taubaté,46344.000000,Espanhol,4,10,1980,4,3,False,False,3Q_4B,1980s
2,2,Estudos,Graduação,São Bernardo,71613.000000,Ingles,3,59,1991,4,5,False,False,5Q_4B,1990s
3,3,Viagem,Graduação,São Paulo,22253.544503,Espanhol,4,22,2010,1,1,False,True,1Q_1B,2010s
4,4,Trabalho,Doutorado,São Paulo,58293.000000,Ingles,3,8,2007,3,4,False,False,4Q_3B,2000s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2235,Trabalho,Graduação,São Paulo,60765.025000,Espanhol e Ingles,4,12,1993,4,4,False,True,4Q_4B,1990s
2236,2236,Estudos,Doutorado,Campinas,67264.636364,Mandarim,3,90,1972,4,5,False,True,5Q_4B,<1980
2237,2237,Viagem,Graduação,São Paulo,56981.000000,Espanhol e Ingles,3,1,2007,3,4,False,False,4Q_3B,2000s
2238,2238,Trabalho,Mestrado,São Paulo,69245.000000,Ingles,4,14,1982,3,5,False,False,5Q_3B,1980s


In [6]:
df.describe()

,ID_Cliente,Renda_anual,NPS,Num_Aulas,Ano_de_nascimento,Banheiros_na_casa,Quartos_na_casa
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,1119.500000,51897.064278,2.836161,20.305357,1994.805804,2.996429,3.473214
std,646.776623,21113.972662,1.200876,19.818528,11.984069,1.404703,1.719290
min,0.000000,1730.000000,1.000000,1.000000,1919.000000,1.000000,1.000000
25%,559.750000,35783.750000,2.000000,8.000000,1985.000000,2.000000,2.000000
50%,1119.500000,51789.500000,3.000000,14.000000,1996.000000,3.000000,3.000000
75%,1679.250000,68179.500000,4.000000,26.000000,2003.000000,4.000000,5.000000
max,2239.000000,162397.000000,5.000000,99.000000,2022.000000,5.000000,6.000000


# Analise Macro

#### PERFIL E SEGMENTAÇÃO DE CLIENTES